In [1]:
import urllib
import sqlalchemy
import numpy as np
import pandas as pd
from datetime import datetime

import os
import pandas as pd
import numpy as np


from helpers import rutas, validarProduccion
rutaRelativa        = rutas.rutas()
print(rutaRelativa)
estadoProduccion    = validarProduccion.validarProduccion()


from classes import dbAzure, email
dbAzure = dbAzure.DBAzure(estadoProduccion)
Email   = email.Email

print(estadoProduccion)

/Users/camilo/Desktop/PROYECTOS
CONEXION CON EL SERVIDOR
True


In [2]:
# CREAR HISTORICO FRIO DS
def queryCreateTableHistorico_equipos_DS(conection_DB, nombre_tabla, dataframe):
    
    # ELIMINA BASE DE DATOS
    conection_DB.connection.execute(f'DROP TABLE GestionProcesosTienda.dbo.{nombre_tabla};')
    conection_DB.connection.commit()
    
    # CREA ARREGLO CON LAS COLUMNAS Y SUS TIPOS
    columns = []
    for index in dataframe.dtypes.index:
        tipo = str(dataframe.dtypes.loc[index])
        if tipo == "object":
            tipo = 'varchar'
        elif tipo == "int64":
            tipo = 'int'
        elif tipo == "float64":
            tipo = 'float'
        columns.append({'name': index, 'type': tipo})
    
    print(columns)
    
    # DA FORMATO A QUERY PARA CREAR TABLA 
    query = f"CREATE TABLE GestionProcesosTienda.dbo.{nombre_tabla} (\nid bigint IDENTITY(1,1) NOT NULL,"
    for column in columns:
        if column['type'] == 'varchar':
            query += f'\n{column["name"]} varchar(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL,'
        elif column['type'] == 'int':
            query += f'\n{column["name"]} bigint NOT NULL,'
    query += f"\ncreated_at datetime NOT NULL, \nupdated_at datetime NOT NULL, \nCONSTRAINT PK_{nombre_tabla}_3213E83F1717089E PRIMARY KEY (id)\n);"

    # SI LA TABLA NO EXISTE EN LA BASE DE DATOS LA CREA
    tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
    if 'historico_equipos_frio_DS' not in tablas.TABLE_NAME.unique():
        # CREA TABLA
        conection_DB.connection.execute(query)
        conection_DB.connection.commit()

        # AGREGA LAS COLUMNAS DE FECHA DE CREACION Y ACTUALIZACION
        dataframe['created_at']   = datetime.today()
        dataframe['updated_at']   = datetime.today()

        # INSERTA DATOS EN BASE DE DATOS
        db_params               = urllib.parse.quote_plus(conection_DB.dataConnection)
        engine                  = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
        dataframe.to_sql('historico_equipos_frio_DS', con=engine, index=False, if_exists="append", schema="dbo")

In [3]:
display(f'{rutaRelativa}/ROTACION/temperatura_diarias.cvs')

'/Users/camilo/Desktop/PROYECTOS/ROTACION/temperatura_diarias.cvs'

In [4]:
dataframe = pd.read_csv(f'{rutaRelativa}/ROTACION/temperatura_diarias.csv')
dataframe = dataframe
#dataframe = dataframe.drop(['EAN','SECCION','ALMACENAMIENTO','Fecha Merma','SECCION','N° Doc. Merma','ZONA','Creado por','Modificado','Creado','NO_CONFORMIDAD','CANTIDAD','ESTADO_RETIRO','FECHA_RETIRO','DESTINO','MES','ID_SECCION','LIMPIEZA','FECHA_SOLICITUD','PROVEEDOR','UBICACION','FECHA_VENCIMIENTO' ], axis=1)
dataframe = dataframe.replace({np.nan: 'None'})
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' + x if(int(x) < 10) else x)

# MODIFICAR NOMBRES DE COLUMAS
dataframe = dataframe.drop('AC2', axis=1)

dataframe = dataframe.rename({
    'Creado por'    :'Creado_por'
    },axis=1)

#for i in dataframe.index:
#    print(dataframe['Creado'])
#
#for i in dataframe.index:
#    print(dataframe.iloc[1]['Creado'])
#    dataframe.iloc[i]['ANO'] = i+1
#    print(dataframe.iloc[i]['ANO'])

dataframe['ANO'] = 2020;
dataframe['MES'] = 1;
dataframe['DIA'] = 27;
dataframe['CAMBIO_ESTADO'] = 'None'
dataframe['nombreProducto2'] = ''
dataframe['temperaturaProducto2'] = ''
dataframe['CANTIDAD_FALLOS'] = '0'

dataframe['nombreMaquina'] = dataframe["nombreMaquina"].apply(lambda x: pd.Series(str(x).split(".")[0]))

display(dataframe)
dataframe.dtypes

,Local,nombreMaquina,NUMERO_SERIE,Temperatura,Creado_por,HORA,FechaReal,FECHA,nombreProducto,temperaturaProducto,...,ZONA,EQUIPO,ESTADO,ANO,MES,DIA,CAMBIO_ESTADO,nombreProducto2,temperaturaProducto2,CANTIDAD_FALLOS
0,O716 - SPID BILBAO,16,None,5,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
1,O716 - SPID BILBAO,17,None,1,"Piedrahita Ortiz,Mauricio",17:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
2,O716 - SPID BILBAO,17,None,1,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
3,O716 - SPID BILBAO,18,None,1,"Piedrahita Ortiz,Mauricio",17:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
4,O716 - SPID BILBAO,18,None,1,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10946,CHJ5028800 - DARKSTORE KENNEDY,3,sin n°de serie,None,robotrocket1,09:00,02/03/2023,2 marzo 2023,None,None,...,Fiambres y Quesos,Equipo Refrigerado,None,2020,1,27,None,,,0
10947,CHJ5028800 - DARKSTORE KENNEDY,4,sin n°de serie,None,robotrocket1,09:00,02/03/2023,2 marzo 2023,None,None,...,Fiambres y Quesos,Equipo Refrigerado,None,2020,1,27,None,,,0
10948,None,None,None,None,robotrocket1,None,None,None,None,None,...,None,None,NO OPERATIVO,2020,1,27,None,,,0
10949,CHJ5028800 - DARKSTORE KENNEDY,1,None,None,robotrocket1,None,None,None,None,None,...,None,None,NO OPERATIVO,2020,1,27,None,,,0


Local                   object
nombreMaquina           object
NUMERO_SERIE            object
Temperatura             object
Creado_por              object
HORA                    object
FechaReal               object
FECHA                   object
nombreProducto          object
temperaturaProducto     object
AC1                     object
encargadoAC             object
TIPO_EQUIPO             object
Creado                  object
ZONA                    object
EQUIPO                  object
ESTADO                  object
ANO                      int64
MES                      int64
DIA                      int64
CAMBIO_ESTADO           object
nombreProducto2         object
temperaturaProducto2    object
CANTIDAD_FALLOS         object
dtype: object

In [5]:
display(dataframe)
dataframe.dtypes

,Local,nombreMaquina,NUMERO_SERIE,Temperatura,Creado_por,HORA,FechaReal,FECHA,nombreProducto,temperaturaProducto,...,ZONA,EQUIPO,ESTADO,ANO,MES,DIA,CAMBIO_ESTADO,nombreProducto2,temperaturaProducto2,CANTIDAD_FALLOS
0,O716 - SPID BILBAO,16,None,5,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
1,O716 - SPID BILBAO,17,None,1,"Piedrahita Ortiz,Mauricio",17:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
2,O716 - SPID BILBAO,17,None,1,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
3,O716 - SPID BILBAO,18,None,1,"Piedrahita Ortiz,Mauricio",17:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
4,O716 - SPID BILBAO,18,None,1,"Piedrahita Ortiz,Mauricio",21:00,None,7 junio 2022,None,None,...,None,None,None,2020,1,27,None,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10946,CHJ5028800 - DARKSTORE KENNEDY,3,sin n°de serie,None,robotrocket1,09:00,02/03/2023,2 marzo 2023,None,None,...,Fiambres y Quesos,Equipo Refrigerado,None,2020,1,27,None,,,0
10947,CHJ5028800 - DARKSTORE KENNEDY,4,sin n°de serie,None,robotrocket1,09:00,02/03/2023,2 marzo 2023,None,None,...,Fiambres y Quesos,Equipo Refrigerado,None,2020,1,27,None,,,0
10948,None,None,None,None,robotrocket1,None,None,None,None,None,...,None,None,NO OPERATIVO,2020,1,27,None,,,0
10949,CHJ5028800 - DARKSTORE KENNEDY,1,None,None,robotrocket1,None,None,None,None,None,...,None,None,NO OPERATIVO,2020,1,27,None,,,0


Local                   object
nombreMaquina           object
NUMERO_SERIE            object
Temperatura             object
Creado_por              object
HORA                    object
FechaReal               object
FECHA                   object
nombreProducto          object
temperaturaProducto     object
AC1                     object
encargadoAC             object
TIPO_EQUIPO             object
Creado                  object
ZONA                    object
EQUIPO                  object
ESTADO                  object
ANO                      int64
MES                      int64
DIA                      int64
CAMBIO_ESTADO           object
nombreProducto2         object
temperaturaProducto2    object
CANTIDAD_FALLOS         object
dtype: object

In [6]:
# EJECUTA FUNCION HISTORITO EQUIPOS FRIO DS
nombre_tabla = 'historico_equipos_frio_DS'
queryCreateTableHistorico_equipos_DS(dbAzure, nombre_tabla, dataframe)

[{'name': 'Local', 'type': 'varchar'}, {'name': 'nombreMaquina', 'type': 'varchar'}, {'name': 'NUMERO_SERIE', 'type': 'varchar'}, {'name': 'Temperatura', 'type': 'varchar'}, {'name': 'Creado_por', 'type': 'varchar'}, {'name': 'HORA', 'type': 'varchar'}, {'name': 'FechaReal', 'type': 'varchar'}, {'name': 'FECHA', 'type': 'varchar'}, {'name': 'nombreProducto', 'type': 'varchar'}, {'name': 'temperaturaProducto', 'type': 'varchar'}, {'name': 'AC1', 'type': 'varchar'}, {'name': 'encargadoAC', 'type': 'varchar'}, {'name': 'TIPO_EQUIPO', 'type': 'varchar'}, {'name': 'Creado', 'type': 'varchar'}, {'name': 'ZONA', 'type': 'varchar'}, {'name': 'EQUIPO', 'type': 'varchar'}, {'name': 'ESTADO', 'type': 'varchar'}, {'name': 'ANO', 'type': 'int'}, {'name': 'MES', 'type': 'int'}, {'name': 'DIA', 'type': 'int'}, {'name': 'CAMBIO_ESTADO', 'type': 'varchar'}, {'name': 'nombreProducto2', 'type': 'varchar'}, {'name': 'temperaturaProducto2', 'type': 'varchar'}, {'name': 'CANTIDAD_FALLOS', 'type': 'varchar'}

/var/folders/rs/5wl8vzfj4vz7xxdjjr9hk3800000gn/T/ipykernel_97280/948984861.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
